# 第 9 章　符合Python风格的对象

# 9.1　对象表示形式

# 9.2　再谈向量类

In [2]:
# vector2d_v0.py
from array import array
import math

class Vector2d:
    typecode = 'd'  # <1>

    def __init__(self, x, y):
        self.x = float(x)    # <2>
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))  # <3>

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)  # <4>

    def __str__(self):
        return str(tuple(self))  # <5>

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +  # <6>
                bytes(array(self.typecode, self)))  # <7>

    def __eq__(self, other):
        return tuple(self) == tuple(other)  # <8>

    def __abs__(self):
        return math.hypot(self.x, self.y)  # <9>

    def __bool__(self):
        return bool(abs(self))  # <10>


In [4]:
v1 = Vector2d(1,2)

In [3]:
Vector2d.typecode

'd'

In [5]:
v1.typecode

'd'

In [8]:
bytes(v1)

b'd\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00@'

In [9]:
v1 == [1,2]

True

## 9.3 备选构造方法

In [12]:
from array import array
import math


class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

# BEGIN VECTOR2D_V1
    @classmethod  # <1>
    def frombytes(cls, octets):  # <2>
        typecode = chr(octets[0])  # <3>
        memv = memoryview(octets[1:]).cast(typecode)  # <4>
        return cls(*memv)  # <5>
# END VECTOR2D_V1

In [13]:
v1_clone = Vector2d.frombytes(bytes(v1))
v1_clone

1.0 2.0


Vector2d(1.0, 2.0)

## 9.4　classmethod与staticmethod

In [14]:
class Demo:
    @classmethod
    def klassmeth(*args):
        return args 
    
    @staticmethod
    def statmeth(*args):
        return args

In [15]:
Demo.klassmeth()

(__main__.Demo,)

In [16]:
Demo.klassmeth('spam')

(__main__.Demo, 'spam')

In [17]:
Demo.statmeth()

()

In [18]:
Demo.statmeth('spam')

('spam',)

## 9.5　格式化显示

In [19]:
format(42, 'b')

'101010'

In [20]:
format(2/3, '.1%')

'66.7%'

In [21]:
from datetime import datetime
now = datetime.now()
format(now, '%H:%M:%S')

'11:50:00'

In [22]:
"It's now {:%I:%M %p}".format(now)

"It's now 11:50 AM"

In [24]:
v1 = Vector2d(3, 4)
format(v1)

'(3.0, 4.0)'

In [26]:
format(v1, '.3f .3f')

TypeError: unsupported format string passed to Vector2d.__format__

In [27]:
from array import array
import math


class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [28]:
format(Vector2d(1, 1), 'p')

'<1.4142135623730951, 0.7853981633974483>'

In [29]:
format(Vector2d(1, 1), '.3ep')

'<1.414e+00, 7.854e-01>'

In [30]:
format(Vector2d(1, 1), '0.5fp')

'<1.41421, 0.78540>'

## 9.6　可散列的Vector2d

In [31]:
from array import array
import math

# BEGIN VECTOR2D_V3_PROP
class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)  # <1>
        self.__y = float(y)

    @property  # <2>
    def x(self):  # <3>
        return self.__x  # <4>

    @property  # <5>
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))  # <6>

    # remaining methods follow (omitted in book listing)
# END VECTOR2D_V3_PROP

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

# BEGIN VECTOR_V3_HASH
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)
# END VECTOR_V3_HASH

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    def angle(self):
        return math.atan2(self.y, self.x)

    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [32]:
v1 = Vector2d(3, 4)
hash(v1)

7

In [33]:
v2 = Vector2d(3.1, 4.2)
hash(v2)

384307168202284039

In [34]:
set([v1, v2])

{Vector2d(3.0, 4.0), Vector2d(3.1, 4.2)}

## 9.7　Python的私有属性和“受保护的”属性

In [41]:
v1 = Vector2d(3, 4)
v1.__dict__,v1

({'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}, Vector2d(3.0, 4.0))

In [42]:
v1._Vector2d__x = 4

In [43]:
v1

Vector2d(4, 4.0)

总之，Vector2d 的分量都是“私有的”，而且 Vector2d 实例都是“不 可变的”。我用了两对引号，这是因为并不能真正实现私有和不可 变。

## 9.8　使用 __slots__ 类属性节省空间

## 9.9　覆盖类属性

In [46]:
v1 = Vector2d(1.1, 2.2)
dumpd = bytes(v1)
dumpd

b'd\x9a\x99\x99\x99\x99\x99\xf1?\x9a\x99\x99\x99\x99\x99\x01@'

In [47]:
v1.typecode = 'f'
dumpf = bytes(v1)
dumpf

b'f\xcd\xcc\x8c?\xcd\xcc\x0c@'

In [48]:
Vector2d.typecode

'd'